<h1>Beocat Tutorial</h1>
Navigate to the following links for more info:

Linux Basics - https://support.beocat.ksu.edu/BeocatDocs/index.php?title=LinuxBasics </br>
Basic Slurm - https://support.beocat.ksu.edu/BeocatDocs/index.php?title=SlurmBasics </br>
Advanced Slurm - https://support.beocat.ksu.edu/BeocatDocs/index.php?title=AdvancedSlurm</br>
Running Python Scripts - https://support.beocat.ksu.edu/BeocatDocs/index.php?title=Installed_software#Python

# Setting up Beocat Account
In order to submit jobs to beocat, you must first set up an account. This can be done here:
https://account.beocat.ksu.edu/

# Getting files onto Beocat

In order to run our training scripts on Beocat we must transfer them to Beocat. In my experience, the easiest way to do this is through SFTP (Secure File Transfer Protocol). If you prefer a GUI, applications like FileZilla will be more than sufficient. Navigate to the directory from which you would like to transfer files and type the following commands:

In [ ]:
> sftp myusername@headnode.beocat.ksu.edu
> Password: (password will show no response on screen)
Connected to headnode.beocat.ksu.edu.
> sftp> put myfile.txt
Uploading myfile.txt to /homes/myusername/myfile.txt
myfile.txt  100%    0     0.0KB/s   00:00
> sftp> exit

Oftentimes you will want to transfer an entire directory to beocat. This is done with the -r flag (for "recursive"):

In [ ]:
> sftp> put -r mydirectory

## SSH Into Beocat

Now we will SSH into beocat so we can run commands. This can be accomplished with the command:

In [ ]:
ssh username@headnode.beocat.ksu.edu

You'll know you are successfully logged in when you see a prompt that says:

In [ ]:
(machinename:~) username%

## Configuring Python
Now we need to do some configuration in order to run out python script. The first thing we must do is pick a Python module. In order to list all the available modules, run:

In [ ]:
> module avail Python/

We can then pick a module using:

In [ ]:
module load Python/SOME_VERSION_THAT_YOU_PICKED_FROM_THE_LIST

The next thing we have to do is create a virtual environment so we can install packages. Complete the following steps in order to accomplish this:

In [ ]:
# create a new directory for our virtualenvs
> mkdir ~/virtualenvs
> cd ~/virtualenvs

# create a new virtualenv called test
> virtualenv test

# verify that the virtualenv is created
> ls ~/virtualenvs

# activate the virtualenv
> source ~/virtualenvs/test/bin/activate

Now we can install any packages we want using pip. Keep in mind that <b>everytime you log in you will have to load the python module and activate the virtualenv</b>

## Run Job Script
In order to submit a job to Beocat, it <b>must be contained in a shell script</b>. If you are trying to run a python script, it is very easy to run through a shell script. Here is an example:

In [ ]:
#!/bin/bash
# run_train_single_agent_basic.sh
# Basic Shell Script to run Python
 
python train_single_agent_basic.py -e 'CartPole-v1' --agent "DQN" --stop-timesteps 30000000 --seed 1 --ram 15 --num_workers 6  --num_envs_per_workers 4

As you can see, the shell file just contains the command you'd use to run the python script in the command line. </br>
<b>Always remember to start your shell scripts with #!/bin/sh, or else slurm will not recognize it</b></br>

Now it it time to submit the job. The following command will go in the command line, inside of the same directory where your shell script is stored: </br>
1. sbatch: command to submit batch script
2. --mem-per-cpu: tells how much memory is needed per core
3. --time: how much runtime is needed. "minutes", "minutes:seconds", "hours:minutes:seconds", "days-hours", "days-hours:minutes" and "days-hours:minutes:seconds"
4. --cpus-per-task: how many cpus are needed
5. --ntasks: how many tasks will be run
6. --nodes: how many machines to be run on
7. --gres=gpus:1: requests one GPU node. More info at: https://support.beocat.ksu.edu/BeocatDocs/index.php?title=AdvancedSlurm#CUDA
8. The shell script that you wish to run

In [ ]:
sbatch --time=1-0 --mem-per-cpu=5gb --cpus-per-task=1 --ntasks=1 --nodes=1 --gres=gpu:1 run_train_single_agent_basic.sh

## Monitoring Your Job
Once your job is running, you can use the following commands to monitor it:

In [ ]:
#Display all of your current jobs
kstat --me

#Detailed information about a certain job
kstat -j 157054

#Display completed job within last # days ex. kstat -d 5
kstat -d #

# Viewing Output
Slurm will output a file to the working directory after the job is done running. This file will be titled something like "slurm-2992962.out". In order to view the contents of this file we can simple run:

In [ ]:
> cat slurm-2992962.out